In [1]:
import zipfile
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
%matplotlib inline
%reload_ext autotime

time: 0 ns (started: 2024-07-26 14:00:30 +09:00)


In [23]:
with zipfile.ZipFile('./data/TL_gps_data.tar', 'r') as zip_ref:
    zip_ref.extractall('./data/TL_gps_data')

time: 7.92 s (started: 2024-07-09 15:56:15 +09:00)


# 여행객에 대한 정보

- **TRAVEL_ID**: 여행 ID
  - 고유값의 개수: 1,118
  - 다른 csv 파일과 연관지어 생각
- **LoDGING_NM**: 숙소명
  - 고유값의 개수 885
  - 소노문 단양: 1.4%
  - 포레스트리솜: 0.9%
  - 소노벨 천안: 0.9%
  - 헤이븐호텔: 0.8%
  - 숙소: 0.8%
- **LODGING_PAYMENT_SEQ**: 숙박경비순번
  - 역시 순번이 정확히 무엇을 의미하는지 모르겠음
  - 1: 57.7%
  - 0: 37.5%
  - 2: 3.8%
  - 3: 0.8%
  - 4: 0.2%
- **LODGING_TYPE_CD**: 숙소유형코드
  - 코드 'HTY'
  - Range: [1 ~ 12]
- **RSVT_YN**: 예약여부
  - Y: 93.9%
  - N     6.1%
- **CHK_OUT_DT_MIN**, **CHK_OUT_DT_MIN**: 체크인아웃시간
  - 전부 NULL 값으로 제외
- **PAYMENT_NUM**: 소비인원
  - 2: 47.0%
  - 4: 16.7%
  - 3: 16.4%
  - 1: 11.9%
  - 5: 3.6%
  - etc.
- **BRNO**: 사업자등록번호
  - 전부 NULL 값으로 제외
- **STORE_NM**: 상호명
  - 고유값의 개수가 400개인데, 아래 top5의 '야놀자'를 보면 알 수 있듯이, 같은 상호명이 다르게 처리되어 있는 경우가 있음
  - 아고다: 6.9%
  - (주)야놀자: 6.0%
  - 에어비앤비: 5.4%
  - (주)여기어때컴퍼니: 4.3%
  - ㈜야놀자: 3.7%
- **ROAD_NM_ADDR**, **LOTNO_ADDR**: 도로명 주소, 지번 주소
  - 대부분 **STORE_NM**과 같이 따라감('아고다'처럼 아닌 것도 있으니 주의)
  - 확인 결과, '(주)야놀자'와 '㈜야놀자'는 같은 상호명으로 도로명과 지번 주소가 같음
- **ROAD_NM_CD**, **LOTNO_CD**: 도로명코드, 지번코드
  - **ROAD_NM_ADDR**과 같이 따라갈 것으로 생각했으나, 확인 결과 아님
- **PAYMENT_DT**: 결제일시_분
  - {YYYY}-{MM}-{DD} HH:MM
  - 간단한 처리후 가장 많은 달을 찾아보니, 역시 5, 6, 8, 7 로 여행을 많이 가는 달과 일치
- **PAYMENT_MTHD_SE**: 결제방식구분
  - 코드 'PAY'
  - Range: [1 ~ 5]
- **PAYMENT_AMT_WON**: 결제금액_원
  - 1원 단위로 떨어지며, 0이 NULL 값일 가능성 다분
  - 0         5.5%
  - 50000     3.3%
  - 80000     2.6%
  - 60000     2.6%
  - 100000    2.4%
  - etc.
- **PAYMENT_ETC**: 소비내역_기타
  - 고유값의 개수가 428개
  - 밑에 '숙박비'도 그렇고, '숙박비;종량제봉투', '숙박비-디럭스' 등의 값이 있어서 전처리할 내용 엄청 많을 것으로 예상
  - 숙박비: 34.0%
  - 숙박비: 5.1%
  - 호텔 케니 여수: 0.4%
  - Ramada by Wyndham Daejeon: 0.3%
  - 신라스테이 여수: 0.3%
  - etc.

In [17]:
data = pd.read_csv('./data/TL_csv/tn_lodge_consume_his_숙박소비내역_G.csv')

data.head(n=3)

,TRAVEL_ID,LODGING_NM,LODGING_PAYMENT_SEQ,LODGING_TYPE_CD,RSVT_YN,CHK_IN_DT_MIN,CHK_OUT_DT_MIN,PAYMENT_NUM,BRNO,STORE_NM,ROAD_NM_ADDR,LOTNO_ADDR,ROAD_NM_CD,LOTNO_CD,PAYMENT_DT,PAYMENT_MTHD_SE,PAYMENT_AMT_WON,PAYMENT_ETC
0,g_g000950,구례자연드림파크 휴펜션,0,3,Y,NaN,NaN,4,NaN,구례자연드림파크 휴펜션,전남 구례군 용방면 용산로 107-81,전남 구례군 용방면 죽정리 452,NaN,NaN,NaN,4,108000,NaN
1,g_g003956,국립대전숲체원,0,9,Y,NaN,NaN,4,NaN,국립대전숲체원,대전 유성구 숲체원로 124,대전 유성구 성북동 산 1-13,NaN,NaN,NaN,1,66000,NaN
2,g_g005377,낭만서프하우스,0,5,Y,NaN,NaN,2,NaN,낭만서프하우스,전남 고흥군 영남면 해맞이로 680,전남 고흥군 영남면 남열리 173,NaN,NaN,NaN,1,120000,NaN


time: 31 ms (started: 2024-07-11 16:18:17 +09:00)


In [46]:
# 각 값의 백분율을 계산하고 소수 둘째 자리에서 반올림한 후 '%'를 포함하여 문자열로 변환
percentage_counts = (data['PAYMENT_ETC'].value_counts(normalize=True) * 100).round(1).apply(lambda x: f"{x}%")
print(percentage_counts)

PAYMENT_ETC
숙박비                          34.0%
숙박비                           5.1%
호텔 케니 여수                      0.4%
Ramada by Wyndham Daejeon     0.3%
신라스테이 여수                      0.3%
                             ...  
단양 꽃들의 물감찾기 펜션                0.1%
숙박비;종량제봉투                     0.1%
숙박비-디럭스                       0.1%
COMO_HILL D                   0.1%
음성 원남테마공원캠핑장                  0.1%
Name: proportion, Length: 428, dtype: object
time: 0 ns (started: 2024-07-11 16:39:42 +09:00)


In [44]:
pd.to_datetime(data['PAYMENT_DT']).dt.month.value_counts(normalize=True) * 100

PAYMENT_DT
5.0    27.838828
6.0    25.152625
8.0    24.053724
7.0    15.628816
4.0     3.785104
9.0     2.442002
3.0     0.610501
2.0     0.244200
1.0     0.244200
Name: proportion, dtype: float64

time: 0 ns (started: 2024-07-11 16:36:35 +09:00)


In [36]:
data[data['STORE_NM'].isin(['아고다'])]['ROAD_NM_ADDR'].value_counts()

ROAD_NM_ADDR
전북 완주군 이서면 오공로 21-25    1
서울 중구 을지로 100           1
서울 마포구 마포대로 217         1
Name: count, dtype: int64

time: 16 ms (started: 2024-07-11 16:33:03 +09:00)


In [52]:
pd.to_datetime(data['PHOTO_FILE_DT']).dt.month.value_counts()

PHOTO_FILE_DT
9    4236
8    3939
6    2937
7    1817
5    1752
4      51
Name: count, dtype: int64

time: 16 ms (started: 2024-07-10 16:58:19 +09:00)


In [38]:
data['TRAVEL_STATUS_YMD'].str.split('~').apply(lambda x: x[0])

0       2023-06-14
1       2023-07-28
2       2023-09-07
3       2023-07-01
4       2023-07-18
           ...    
2555    2023-08-16
2556    2023-09-10
2557    2023-09-23
2558    2023-09-16
2559    2023-08-15
Name: TRAVEL_STATUS_YMD, Length: 2560, dtype: object

time: 16 ms (started: 2024-07-10 16:32:24 +09:00)


In [44]:
data['MONTH'] = pd.to_datetime(data['TRAVEL_STATUS_YMD'].str.split('~').apply(lambda x: x[0])).dt.month

data['MONTH'].value_counts(normalize=True) * 100 

MONTH
9    27.265625
8    25.664062
6    21.250000
5    13.007812
7    12.148438
4     0.664062
Name: proportion, dtype: float64

time: 16 ms (started: 2024-07-10 16:35:40 +09:00)


In [5]:
tc_sgg = pd.read_csv('./data/TL_csv/tc_sgg_시군구코드.csv')

tc_sgg

,SGG_CD,SGG_CD1,SGG_CD2,SGG_CD3,SGG_CD4,SIDO_NM,SGG_NM,DONG_NM,RI_NM
0,1100000000,11,NaN,NaN,NaN,서울특별시,NaN,NaN,NaN
1,1111000000,11,110.0,NaN,NaN,서울특별시,종로구,NaN,NaN
2,1111010100,11,110.0,101.0,0.0,서울특별시,종로구,청운동,NaN
3,1111010200,11,110.0,102.0,0.0,서울특별시,종로구,신교동,NaN
4,1111010300,11,110.0,103.0,0.0,서울특별시,종로구,궁정동,NaN
...,...,...,...,...,...,...,...,...,...
8070,5013025929,50,130.0,259.0,29.0,제주특별자치도,서귀포시,성산읍,신풍리
8071,5013025930,50,130.0,259.0,30.0,제주특별자치도,서귀포시,성산읍,신천리
8072,5013025931,50,130.0,259.0,31.0,제주특별자치도,서귀포시,성산읍,난산리
8073,5013031000,50,130.0,310.0,0.0,제주특별자치도,서귀포시,안덕면,NaN


time: 31 ms (started: 2024-07-10 16:14:39 +09:00)


In [23]:
tc_sgg[tc_sgg['SGG_CD1'] == 26]

,SGG_CD,SGG_CD1,SGG_CD2,SGG_CD3,SGG_CD4,SIDO_NM,SGG_NM,DONG_NM,RI_NM
493,2600000000,26,NaN,NaN,NaN,부산광역시,NaN,NaN,NaN
494,2611000000,26,110.0,NaN,NaN,부산광역시,중구,NaN,NaN
495,2611010100,26,110.0,101.0,0.0,부산광역시,중구,영주동,NaN
496,2611010200,26,110.0,102.0,0.0,부산광역시,중구,대창동1가,NaN
497,2611010300,26,110.0,103.0,0.0,부산광역시,중구,대창동2가,NaN
...,...,...,...,...,...,...,...,...,...
748,2671025930,26,710.0,259.0,30.0,부산광역시,기장군,일광읍,원리
749,2671025931,26,710.0,259.0,31.0,부산광역시,기장군,일광읍,화전리
750,2671025932,26,710.0,259.0,32.0,부산광역시,기장군,일광읍,횡계리
751,2671025933,26,710.0,259.0,33.0,부산광역시,기장군,일광읍,용천리


time: 16 ms (started: 2024-07-10 16:21:13 +09:00)


In [8]:
tc_sgg['SGG_CD'] = tc_sgg['SGG_CD'].astype('str')

tc_sgg['SGG_CD']

0       1100000000
1       1111000000
2       1111010100
3       1111010200
4       1111010300
           ...    
8070    5013025929
8071    5013025930
8072    5013025931
8073    5013031000
8074    5013032000
Name: SGG_CD, Length: 8075, dtype: object

time: 0 ns (started: 2024-07-10 16:15:35 +09:00)


In [26]:
tc_sgg[tc_sgg['SGG_CD'].str.startswith('42150')]

,SGG_CD,SGG_CD1,SGG_CD2,SGG_CD3,SGG_CD4,SIDO_NM,SGG_NM,DONG_NM,RI_NM
3011,4215000000,42,150.0,NaN,NaN,강원도,강릉시,NaN,NaN
3012,4215010100,42,150.0,101.0,0.0,강원도,강릉시,홍제동,NaN
3013,4215010200,42,150.0,102.0,0.0,강원도,강릉시,남문동,NaN
3014,4215010300,42,150.0,103.0,0.0,강원도,강릉시,명주동,NaN
3015,4215010400,42,150.0,104.0,0.0,강원도,강릉시,성내동,NaN
3016,4215010500,42,150.0,105.0,0.0,강원도,강릉시,임당동,NaN
3017,4215010600,42,150.0,106.0,0.0,강원도,강릉시,금학동,NaN
3018,4215010700,42,150.0,107.0,0.0,강원도,강릉시,용강동,NaN
3019,4215010800,42,150.0,108.0,0.0,강원도,강릉시,성남동,NaN
3020,4215010900,42,150.0,109.0,0.0,강원도,강릉시,옥천동,NaN


time: 16 ms (started: 2024-07-10 16:22:02 +09:00)


In [87]:
tmp = pd.read_csv('./data/TL_csv/tn_travel_여행_G.csv')

tmp

,TRAVEL_ID,TRAVEL_NM,TRAVELER_ID,TRAVEL_PURPOSE,TRAVEL_START_YMD,TRAVEL_END_YMD,MVMN_NM,TRAVEL_PERSONA,TRAVEL_MISSION,TRAVEL_MISSION_CHECK
0,e_e000005,E02,e000005,5;2;23,2023-04-29,2023-05-03,NaN,충청권 방문/충청권 거주/39세 이하/부부/일반미션,5;2;23,5;2;23
1,e_e000043,E03,e000043,10;12;21;27,2023-04-28,2023-05-01,NaN,충남 방문/거주지 구분 없음/39세 이하/특별미션,10;12;21;27,4;6;2
2,e_e000191,G06,e000191,21,2023-04-28,2023-05-01,NaN,충남 방문/거주지 구분 없음/39세 이하/특별미션,21,24;13;27
3,e_e000316,E01,e000316,21;22;24;26;6,2023-05-27,2023-05-30,대중교통 등,충청권 방문/충청권 외 거주/39세 이하/커플/일반미션,21;22;24;26;6,22;26;6
4,e_e000668,E02,e000668,2,2023-05-20,2023-05-21,자가용,충청권 방문/충청권 거주/39세 이하/3대 동반 여행/일반미션,2,2;1;4
...,...,...,...,...,...,...,...,...,...,...
2555,g_g013127,G06,g013127,1;10;2;23,2023-09-27,2023-09-28,NaN,충청권 방문/충청권 거주/39세 이하/나홀로 여행/일반미션,1;10;2;23,1;2;3
2556,g_g013128,G04,g013128,21;22;24;3;6,2023-09-29,2023-09-30,NaN,전남 방문/거주지 구분 없음/39세 이하/특별미션,21;22;24;3;6,6;1;21
2557,g_g013131,G03,g013131,2;21;22;26;4,2023-09-29,2023-09-30,NaN,전남 방문/거주지 구분 없음/39세 이하/특별미션,2;21;22;26;4,4;22;21
2558,h_h001459,G05,h001459,22,2023-07-06,2023-07-08,대중교통 등,충북 방문/거주지 구분 없음/39세 이하/특별미션,22,22;5;26


time: 31 ms (started: 2024-07-09 17:29:17 +09:00)


In [92]:
tmp['TRAVEL_START_YMD'] = pd.to_datetime(tmp['TRAVEL_START_YMD'])
tmp['TRAVEL_END_YMD'] = pd.to_datetime(tmp['TRAVEL_END_YMD'])

tmp['Day_diff'] = (tmp['TRAVEL_END_YMD'] - tmp['TRAVEL_START_YMD'])

time: 0 ns (started: 2024-07-09 17:32:44 +09:00)


In [108]:
tmp['Day_diff'].value_counts(normalize=True) * 100

Day_diff
1 days    49.960938
2 days    36.367188
3 days    12.343750
4 days     1.132812
5 days     0.156250
0 days     0.039062
Name: proportion, dtype: float64

time: 0 ns (started: 2024-07-09 17:43:30 +09:00)


In [101]:
tmp[tmp['Day_diff'] == '1 days']

,TRAVEL_ID,TRAVEL_NM,TRAVELER_ID,TRAVEL_PURPOSE,TRAVEL_START_YMD,TRAVEL_END_YMD,MVMN_NM,TRAVEL_PERSONA,TRAVEL_MISSION,TRAVEL_MISSION_CHECK,Day_diff
4,e_e000668,E02,e000668,2,2023-05-20,2023-05-21,자가용,충청권 방문/충청권 거주/39세 이하/3대 동반 여행/일반미션,2,2;1;4,1 days
5,e_e001033,E03,e001033,21;22;24;26;27,2023-05-27,2023-05-28,자가용,충청권 방문/충청권 외 거주/39세 이하/커플/일반미션,21;22;24;26;27,13;21;22,1 days
6,e_e001240,E03,e001240,1;22;4;9,2023-06-04,2023-06-05,자가용,충청권 방문/충청권 외 거주/39세 이하/나홀로 여행/일반미션,1;22;4;9,23;6;2,1 days
7,e_e001423,G06,e001423,21,2023-05-29,2023-05-30,자가용,충남 방문/거주지 구분 없음/40세 이상/특별미션,21,21;1;6,1 days
9,e_e002195,G04,e002195,1;23,2023-06-04,2023-06-05,대중교통 등,충청권 방문/충청권 외 거주/39세 이하/커플/일반미션,1;23,1;26;3,1 days
...,...,...,...,...,...,...,...,...,...,...,...
2554,g_g013125,G03,g013125,2;22;27;4,2023-09-27,2023-09-28,NaN,전남 방문/거주지 구분 없음/39세 이하/특별미션,2;22;27;4,1;3;12,1 days
2555,g_g013127,G06,g013127,1;10;2;23,2023-09-27,2023-09-28,NaN,충청권 방문/충청권 거주/39세 이하/나홀로 여행/일반미션,1;10;2;23,1;2;3,1 days
2556,g_g013128,G04,g013128,21;22;24;3;6,2023-09-29,2023-09-30,NaN,전남 방문/거주지 구분 없음/39세 이하/특별미션,21;22;24;3;6,6;1;21,1 days
2557,g_g013131,G03,g013131,2;21;22;26;4,2023-09-29,2023-09-30,NaN,전남 방문/거주지 구분 없음/39세 이하/특별미션,2;21;22;26;4,4;22;21,1 days


time: 16 ms (started: 2024-07-09 17:34:45 +09:00)


In [27]:
tmp1 = pd.read_csv('./data/TL_csv/tn_traveller_master_여행객 Master_G.csv')

tmp1

,TRAVELER_ID,RESIDENCE_SGG_CD,GENDER,AGE_GRP,EDU_NM,EDU_FNSH_SE,MARR_STTS,FAMILY_MEMB,JOB_NM,JOB_ETC,...,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_STATUS_RESIDENCE,TRAVEL_STATUS_DESTINATION,TRAVEL_STATUS_ACCOMPANY,TRAVEL_STATUS_YMD,TRAVEL_MOTIVE_1,TRAVEL_MOTIVE_2,TRAVEL_MOTIVE_3,TRAVEL_COMPANIONS_NUM
0,g002178,29,여,20,6,1,1,3,12,NaN,...,6,6,광주광역시,전남,2인 여행(가족 외),2023-06-14~2023-06-15,1,3.0,9.0,1
1,g004485,30,남,40,7,5,1,2,2,NaN,...,7,7,대전광역시,전북,2인 여행(가족 외),2023-07-28~2023-07-29,1,2.0,3.0,1
2,g005167,11,남,20,6,1,1,3,1,NaN,...,1,7,서울특별시,전남,2인 여행(가족 외),2023-09-07~2023-09-09,7,5.0,1.0,1
3,g002887,28,여,30,5,1,2,2,3,NaN,...,2,6,인천광역시,대전,2인 여행(가족 외),2023-07-01~2023-07-02,2,1.0,3.0,1
4,g004189,41,남,20,6,1,1,4,4,NaN,...,6,5,경기도,충남,2인 여행(가족 외),2023-07-18~2023-07-19,2,3.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,g005855,26,여,20,6,1,1,4,3,NaN,...,2,4,부산광역시,전북,2인 여행(가족 외),2023-08-16~2023-08-17,7,3.0,2.0,1
2556,g012396,29,여,50,5,1,2,4,3,NaN,...,3,5,광주광역시,전남,자녀 동반 여행,2023-09-10~2023-09-10,3,1.0,2.0,1
2557,g012655,36,남,30,7,1,2,4,2,NaN,...,2,4,세종특별자치시,충북,자녀 동반 여행,2023-09-23~2023-09-23,7,3.0,1.0,3
2558,g012604,11,남,20,4,1,1,4,12,NaN,...,2,6,서울특별시,대전,2인 여행(가족 외),2023-09-16~2023-09-16,7,2.0,3.0,1


In [172]:
import pandas as pd

pd.set_option('display.max_columns', None)

org = pd.read_csv('./data/TL_csv/tn_traveller_master_여행객 Master_G.csv')

org

,TRAVELER_ID,RESIDENCE_SGG_CD,GENDER,AGE_GRP,EDU_NM,EDU_FNSH_SE,MARR_STTS,FAMILY_MEMB,JOB_NM,JOB_ETC,INCOME,HOUSE_INCOME,TRAVEL_TERM,TRAVEL_NUM,TRAVEL_LIKE_SIDO_1,TRAVEL_LIKE_SGG_1,TRAVEL_LIKE_SIDO_2,TRAVEL_LIKE_SGG_2,TRAVEL_LIKE_SIDO_3,TRAVEL_LIKE_SGG_3,TRAVEL_STYL_1,TRAVEL_STYL_2,TRAVEL_STYL_3,TRAVEL_STYL_4,TRAVEL_STYL_5,TRAVEL_STYL_6,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_STATUS_RESIDENCE,TRAVEL_STATUS_DESTINATION,TRAVEL_STATUS_ACCOMPANY,TRAVEL_STATUS_YMD,TRAVEL_MOTIVE_1,TRAVEL_MOTIVE_2,TRAVEL_MOTIVE_3,TRAVEL_COMPANIONS_NUM
0,g002178,29,여,20,6,1,1,3,12,NaN,2,7.0,3,4,50,50130,50,50110,26,26500,1,1,1,3,3,2,6,6,광주광역시,전남,2인 여행(가족 외),2023-06-14~2023-06-15,1,3.0,9.0,1
1,g004485,30,남,40,7,5,1,2,2,NaN,5,10.0,2,2,48,48840,48,48220,26,26500,1,1,3,6,4,2,7,7,대전광역시,전북,2인 여행(가족 외),2023-07-28~2023-07-29,1,2.0,3.0,1
2,g005167,11,남,20,6,1,1,3,1,NaN,4,7.0,3,7,46,46710,45,45190,47,47170,2,3,1,4,6,3,1,7,서울특별시,전남,2인 여행(가족 외),2023-09-07~2023-09-09,7,5.0,1.0,1
3,g002887,28,여,30,5,1,2,2,3,NaN,5,8.0,2,1,42,42150,42,42210,28,28710,2,2,3,2,3,5,2,6,인천광역시,대전,2인 여행(가족 외),2023-07-01~2023-07-02,2,1.0,3.0,1
4,g004189,41,남,20,6,1,1,4,4,NaN,4,9.0,2,2,42,42150,50,50110,26,26350,2,1,2,4,6,5,6,5,경기도,충남,2인 여행(가족 외),2023-07-18~2023-07-19,2,3.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,g005855,26,여,20,6,1,1,4,3,NaN,4,4.0,3,6,47,47130,26,26710,50,50130,1,1,1,5,2,1,2,4,부산광역시,전북,2인 여행(가족 외),2023-08-16~2023-08-17,7,3.0,2.0,1
2556,g012396,29,여,50,5,1,2,4,3,NaN,4,8.0,2,3,46,46710,50,50130,46,46880,5,2,3,2,1,7,3,5,광주광역시,전남,자녀 동반 여행,2023-09-10~2023-09-10,3,1.0,2.0,1
2557,g012655,36,남,30,7,1,2,4,2,NaN,6,6.0,3,15,44,44180,44,44825,44,44150,2,2,2,4,6,4,2,4,세종특별자치시,충북,자녀 동반 여행,2023-09-23~2023-09-23,7,3.0,1.0,3
2558,g012604,11,남,20,4,1,1,4,12,NaN,1,6.0,3,5,50,50130,47,47130,26,26500,5,2,2,4,6,4,2,6,서울특별시,대전,2인 여행(가족 외),2023-09-16~2023-09-16,7,2.0,3.0,1


In [173]:
def transform_traveler_id(traveler_id):
    return f'{traveler_id[0]}_{traveler_id}'

org['TRAVEL_ID'] = org['TRAVELER_ID'].apply(transform_traveler_id)

org

,TRAVELER_ID,RESIDENCE_SGG_CD,GENDER,AGE_GRP,EDU_NM,EDU_FNSH_SE,MARR_STTS,FAMILY_MEMB,JOB_NM,JOB_ETC,INCOME,HOUSE_INCOME,TRAVEL_TERM,TRAVEL_NUM,TRAVEL_LIKE_SIDO_1,TRAVEL_LIKE_SGG_1,TRAVEL_LIKE_SIDO_2,TRAVEL_LIKE_SGG_2,TRAVEL_LIKE_SIDO_3,TRAVEL_LIKE_SGG_3,TRAVEL_STYL_1,TRAVEL_STYL_2,TRAVEL_STYL_3,TRAVEL_STYL_4,TRAVEL_STYL_5,TRAVEL_STYL_6,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_STATUS_RESIDENCE,TRAVEL_STATUS_DESTINATION,TRAVEL_STATUS_ACCOMPANY,TRAVEL_STATUS_YMD,TRAVEL_MOTIVE_1,TRAVEL_MOTIVE_2,TRAVEL_MOTIVE_3,TRAVEL_COMPANIONS_NUM,TRAVEL_ID
0,g002178,29,여,20,6,1,1,3,12,NaN,2,7.0,3,4,50,50130,50,50110,26,26500,1,1,1,3,3,2,6,6,광주광역시,전남,2인 여행(가족 외),2023-06-14~2023-06-15,1,3.0,9.0,1,g_g002178
1,g004485,30,남,40,7,5,1,2,2,NaN,5,10.0,2,2,48,48840,48,48220,26,26500,1,1,3,6,4,2,7,7,대전광역시,전북,2인 여행(가족 외),2023-07-28~2023-07-29,1,2.0,3.0,1,g_g004485
2,g005167,11,남,20,6,1,1,3,1,NaN,4,7.0,3,7,46,46710,45,45190,47,47170,2,3,1,4,6,3,1,7,서울특별시,전남,2인 여행(가족 외),2023-09-07~2023-09-09,7,5.0,1.0,1,g_g005167
3,g002887,28,여,30,5,1,2,2,3,NaN,5,8.0,2,1,42,42150,42,42210,28,28710,2,2,3,2,3,5,2,6,인천광역시,대전,2인 여행(가족 외),2023-07-01~2023-07-02,2,1.0,3.0,1,g_g002887
4,g004189,41,남,20,6,1,1,4,4,NaN,4,9.0,2,2,42,42150,50,50110,26,26350,2,1,2,4,6,5,6,5,경기도,충남,2인 여행(가족 외),2023-07-18~2023-07-19,2,3.0,1.0,1,g_g004189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,g005855,26,여,20,6,1,1,4,3,NaN,4,4.0,3,6,47,47130,26,26710,50,50130,1,1,1,5,2,1,2,4,부산광역시,전북,2인 여행(가족 외),2023-08-16~2023-08-17,7,3.0,2.0,1,g_g005855
2556,g012396,29,여,50,5,1,2,4,3,NaN,4,8.0,2,3,46,46710,50,50130,46,46880,5,2,3,2,1,7,3,5,광주광역시,전남,자녀 동반 여행,2023-09-10~2023-09-10,3,1.0,2.0,1,g_g012396
2557,g012655,36,남,30,7,1,2,4,2,NaN,6,6.0,3,15,44,44180,44,44825,44,44150,2,2,2,4,6,4,2,4,세종특별자치시,충북,자녀 동반 여행,2023-09-23~2023-09-23,7,3.0,1.0,3,g_g012655
2558,g012604,11,남,20,4,1,1,4,12,NaN,1,6.0,3,5,50,50130,47,47130,26,26500,5,2,2,4,6,4,2,6,서울특별시,대전,2인 여행(가족 외),2023-09-16~2023-09-16,7,2.0,3.0,1,g_g012604


In [174]:
t = pd.read_csv('./data/TL_csv/tc_codeb_코드B.csv')

t

,idx,cd_a,cd_b,cd_nm,cd_memo,cd_memo2,del_flag,order_num,ins_dt,edit_dt
0,1055,ACT,1,취식,NaN,NaN,N,10,2022-07-05 11:34:29,NaN
1,1056,ACT,2,쇼핑 / 구매,(아이 쇼핑 포함)<br>* 이전 방문지에서 해당 방문지 이동 중 구매한 내역 포함,NaN,N,20,2022-07-05 11:49:08,NaN
2,1057,ACT,3,체험 활동 / 입장 및 관람,NaN,NaN,N,30,2022-07-05 11:49:37,NaN
3,1058,ACT,4,단순 구경 / 산책 / 걷기,NaN,NaN,N,40,2022-07-05 11:49:43,NaN
4,1059,ACT,5,휴식,NaN,NaN,N,50,2022-07-05 11:49:48,NaN
...,...,...,...,...,...,...,...,...,...,...
233,1010,VIS,8,지역 축제/행사,NaN,NaN,N,80,2022-07-05 11:24:15,NaN
234,1011,VIS,9,"역, 터미널, 고속도로 휴게소",NaN,NaN,N,90,2022-07-05 11:24:23,NaN
235,1012,VIS,10,상점,NaN,NaN,N,100,2022-07-05 11:24:31,NaN
236,1013,VIS,11,식당/카페,NaN,NaN,N,110,2022-07-05 11:24:40,NaN


In [175]:
import re

# 매핑 리스트
mapping_list = {
    'EDU_NM': 'EDU', 'EDU_FNSH_SE': 'EFS', 'MARR_STTS': 'MAR', 'JOB_NM': 'JOB', 'JOB_ETC': 'JOE',
    'INCOME': 'INC', 'HOUSE_INCOME': 'INC', 'TRAVEL_TERM': 'TTM'
}

# TRAVEL_STYL_로 시작하는 모든 열을 추가
travel_styl_columns = [col for col in org.columns if re.match(r'TRAVEL_STYL_\d+', col)]
for col in travel_styl_columns:
    mapping_list[col] = 'TSY'
    
# TRAVEL_MOTIVE_로 시작하는 모든 열을 추가
travel_motive_columns = [col for col in org.columns if re.match(r'TRAVEL_MOTIVE_\d+', col)]
for col in travel_motive_columns:
    mapping_list[col] = 'TMT'

# t 데이터 프레임에서 필요한 필터링을 한 번 수행
filtered_t = t[t['cd_a'].isin(mapping_list.values())][['cd_a', 'cd_b', 'cd_nm']]
filtered_t['cd_b'] = filtered_t['cd_b'].astype(int)

# 매핑 딕셔너리 생성
mapping_dicts = {
    cd_a_value: filtered_t[filtered_t['cd_a'] == cd_a_value].set_index('cd_b')['cd_nm'].to_dict()
    for cd_a_value in mapping_list.values()
}

# 각 매핑에 대해 딕셔너리를 생성하고 매핑 적용
for column, cd_a_value in mapping_list.items():
    org[column] = org[column].map(mapping_dicts[cd_a_value])

In [176]:
org

,TRAVELER_ID,RESIDENCE_SGG_CD,GENDER,AGE_GRP,EDU_NM,EDU_FNSH_SE,MARR_STTS,FAMILY_MEMB,JOB_NM,JOB_ETC,INCOME,HOUSE_INCOME,TRAVEL_TERM,TRAVEL_NUM,TRAVEL_LIKE_SIDO_1,TRAVEL_LIKE_SGG_1,TRAVEL_LIKE_SIDO_2,TRAVEL_LIKE_SGG_2,TRAVEL_LIKE_SIDO_3,TRAVEL_LIKE_SGG_3,TRAVEL_STYL_1,TRAVEL_STYL_2,TRAVEL_STYL_3,TRAVEL_STYL_4,TRAVEL_STYL_5,TRAVEL_STYL_6,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_STATUS_RESIDENCE,TRAVEL_STATUS_DESTINATION,TRAVEL_STATUS_ACCOMPANY,TRAVEL_STATUS_YMD,TRAVEL_MOTIVE_1,TRAVEL_MOTIVE_2,TRAVEL_MOTIVE_3,TRAVEL_COMPANIONS_NUM,TRAVEL_ID
0,g002178,29,여,20,대학교(4년제 이상),졸업,미혼,3,학생,NaN,월평균 100만원 미만,월평균 500만원 ~ 600만원 미만,1년,4,50,50130,50,50110,26,26500,자연 선호 매우선호,자연 선호 매우선호,자연 선호 매우선호,자연 선호 약간선호,자연 선호 약간선호,자연 선호 중간선호,도시 선호 중간선호,도시 선호 중간선호,광주광역시,전남,2인 여행(가족 외),2023-06-14~2023-06-15,"일상적인 환경 및 역할에서의 탈출, 지루함 탈피",여행 동반자와의 친밀감 및 유대감 증진,"특별한 목적(칠순여행, 신혼여행, 수학여행, 인센티브여행)",1,g_g002178
1,g004485,30,남,40,대학원 석사 과정,수료,미혼,2,전문가 및 관련 종사자,NaN,월평균 300만원 ~ 400만원 미만,월평균 800만원 ~ 900만원 미만,한달,2,48,48840,48,48220,26,26500,자연 선호 매우선호,자연 선호 매우선호,자연 선호 약간선호,도시 선호 중간선호,중립,자연 선호 중간선호,도시 선호 매우선호,도시 선호 매우선호,대전광역시,전북,2인 여행(가족 외),2023-07-28~2023-07-29,"일상적인 환경 및 역할에서의 탈출, 지루함 탈피","쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",여행 동반자와의 친밀감 및 유대감 증진,1,g_g004485
2,g005167,11,남,20,대학교(4년제 이상),졸업,미혼,3,관리자,NaN,월평균 200만원 ~ 300만원 미만,월평균 500만원 ~ 600만원 미만,1년,7,46,46710,45,45190,47,47170,자연 선호 중간선호,자연 선호 약간선호,자연 선호 매우선호,중립,도시 선호 중간선호,자연 선호 약간선호,자연 선호 매우선호,도시 선호 매우선호,서울특별시,전남,2인 여행(가족 외),2023-09-07~2023-09-09,새로운 경험 추구,SNS 사진 등록 등 과시,"일상적인 환경 및 역할에서의 탈출, 지루함 탈피",1,g_g005167
3,g002887,28,여,30,대학(4년제 미만),졸업,기혼,2,사무 종사자,NaN,월평균 300만원 ~ 400만원 미만,월평균 600만원 ~ 700만원 미만,한달,1,42,42150,42,42210,28,28710,자연 선호 중간선호,자연 선호 중간선호,자연 선호 약간선호,자연 선호 중간선호,자연 선호 약간선호,도시 선호 약간선호,자연 선호 중간선호,도시 선호 중간선호,인천광역시,대전,2인 여행(가족 외),2023-07-01~2023-07-02,"쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식","일상적인 환경 및 역할에서의 탈출, 지루함 탈피",여행 동반자와의 친밀감 및 유대감 증진,1,g_g002887
4,g004189,41,남,20,대학교(4년제 이상),졸업,미혼,4,서비스 종사자,NaN,월평균 200만원 ~ 300만원 미만,월평균 700만원 ~ 800만원 미만,한달,2,42,42150,50,50110,26,26350,자연 선호 중간선호,자연 선호 매우선호,자연 선호 중간선호,중립,도시 선호 중간선호,도시 선호 약간선호,도시 선호 중간선호,도시 선호 약간선호,경기도,충남,2인 여행(가족 외),2023-07-18~2023-07-19,"쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",여행 동반자와의 친밀감 및 유대감 증진,"일상적인 환경 및 역할에서의 탈출, 지루함 탈피",1,g_g004189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,g005855,26,여,20,대학교(4년제 이상),졸업,미혼,4,사무 종사자,NaN,월평균 200만원 ~ 300만원 미만,월평균 200만원 ~ 300만원 미만,1년,6,47,47130,26,26710,50,50130,자연 선호 매우선호,자연 선호 매우선호,자연 선호 매우선호,도시 선호 약간선호,자연 선호 중간선호,자연 선호 매우선호,자연 선호 중간선호,중립,부산광역시,전북,2인 여행(가족 외),2023-08-16~2023-08-17,새로운 경험 추구,여행 동반자와의 친밀감 및 유대감 증진,"쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",1,g_g005855
2556,g012396,29,여,50,대학(4년제 미만),졸업,기혼,4,사무 종사자,NaN,월평균 200만원 ~ 300만원 미만,월평균 600만원 ~ 700만원 미만,한달,3,46,46710,50,50130,46,46880,도시 선호 약간선호,자연 선호 중간선호,자연 선호 약간선호,자연 선호 중간선호,자연 선호 매우선호,도시 선호 매우선호,자연 선호 약간선호,도시 선호 약간선호,광주광역시,전남,자녀 동반 여행,2023-09-10~2023-09-10,여행 동반자와의 친밀감 및 유대감 증진,"일상적인 환경 및 역할에서의 탈출, 지루함 탈피","쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",1,g_g012396
2557,g012655,36,남,30,대학원 석사 과정,졸업,기혼,4,전문가 및 관련 종사자,NaN,월평균 400만원 ~ 500만원 미만,월평균 400만원 ~ 500만원 미만,1년,15,44,44180,44,44825,44,44150,자연 선호 중간선호,자연 선호 중간선호,자연 선호 중간선호,중립,도시 선호 중간선호,중립,자연 선호 중간선호,중립,세종특별자치시,충북,자녀 동반 여행,2023-09-23~2023-09-23,새로운 경험 추구,여행 동반자와의 친밀감 및 유대감 증진,"일상적인 환경 및 역할에서의 탈출, 지루함 탈피",3,g_g012655
2558,g012604,11,남,20,고등학교,졸업,미혼,4,학생,NaN,소득없음,월평균 400만원 ~ 500만원 미만,1년,5,50,50130,47,47130,26,26500,도시 선호 약간선호,자연 선호 중간선호,자연 선호 중간선호,중립,도시 선호 중간선호,중립,자연 선호 중간선호,도시 선호 중간선호,서울특별시,대전,2인 여행(가족 외),2023-09-16~2023-09-16,새로운 경험 추구,"쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",여행 동반자와의 친밀감 및 유대감 증진,1,g_g012604


In [40]:
org.columns

Index(['TRAVELER_ID', 'RESIDENCE_SGG_CD', 'GENDER', 'AGE_GRP', 'EDU_NM',
       'EDU_FNSH_SE', 'MARR_STTS', 'FAMILY_MEMB', 'JOB_NM', 'JOB_ETC',
       'INCOME', 'HOUSE_INCOME', 'TRAVEL_TERM', 'TRAVEL_NUM',
       'TRAVEL_LIKE_SIDO_1', 'TRAVEL_LIKE_SGG_1', 'TRAVEL_LIKE_SIDO_2',
       'TRAVEL_LIKE_SGG_2', 'TRAVEL_LIKE_SIDO_3', 'TRAVEL_LIKE_SGG_3',
       'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4',
       'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
       'TRAVEL_STATUS_RESIDENCE', 'TRAVEL_STATUS_DESTINATION',
       'TRAVEL_STATUS_ACCOMPANY', 'TRAVEL_STATUS_YMD', 'TRAVEL_MOTIVE_1',
       'TRAVEL_MOTIVE_2', 'TRAVEL_MOTIVE_3', 'TRAVEL_COMPANIONS_NUM'],
      dtype='object')

In [64]:
select_columns = ['GENDER', 'AGE_GRP', 'TRAVEL_LIKE_SGG_1', 'TRAVEL_STATUS_DESTINATION', 'TRAVEL_STATUS_ACCOMPANY', 'TRAVEL_STYL_1']

data = org[select_columns]

data

,GENDER,AGE_GRP,TRAVEL_LIKE_SGG_1,TRAVEL_STATUS_DESTINATION,TRAVEL_STATUS_ACCOMPANY,TRAVEL_STYL_1
0,여,20,50130,전남,2인 여행(가족 외),1
1,남,40,48840,전북,2인 여행(가족 외),1
2,남,20,46710,전남,2인 여행(가족 외),2
3,여,30,42150,대전,2인 여행(가족 외),2
4,남,20,42150,충남,2인 여행(가족 외),2
...,...,...,...,...,...,...
2555,여,20,47130,전북,2인 여행(가족 외),1
2556,여,50,46710,전남,자녀 동반 여행,5
2557,남,30,44180,충북,자녀 동반 여행,2
2558,남,20,50130,대전,2인 여행(가족 외),5


In [67]:
mapping_list = {'EDU': ['EDU_NM'], 'EFS': ['EDU_FNSH_SE']}

In [84]:
def get_keys_from_list_value(d, target):
    return [k for k, v in d.items() if target in v]

get_keys_from_list_value()

['EDU']

In [65]:
pd.read_csv('./data/TL_csv/tc_codeb_코드B.csv')

,idx,cd_a,cd_b,cd_nm,cd_memo,cd_memo2,del_flag,order_num,ins_dt,edit_dt
0,1055,ACT,1,취식,NaN,NaN,N,10,2022-07-05 11:34:29,NaN
1,1056,ACT,2,쇼핑 / 구매,(아이 쇼핑 포함)<br>* 이전 방문지에서 해당 방문지 이동 중 구매한 내역 포함,NaN,N,20,2022-07-05 11:49:08,NaN
2,1057,ACT,3,체험 활동 / 입장 및 관람,NaN,NaN,N,30,2022-07-05 11:49:37,NaN
3,1058,ACT,4,단순 구경 / 산책 / 걷기,NaN,NaN,N,40,2022-07-05 11:49:43,NaN
4,1059,ACT,5,휴식,NaN,NaN,N,50,2022-07-05 11:49:48,NaN
...,...,...,...,...,...,...,...,...,...,...
233,1010,VIS,8,지역 축제/행사,NaN,NaN,N,80,2022-07-05 11:24:15,NaN
234,1011,VIS,9,"역, 터미널, 고속도로 휴게소",NaN,NaN,N,90,2022-07-05 11:24:23,NaN
235,1012,VIS,10,상점,NaN,NaN,N,100,2022-07-05 11:24:31,NaN
236,1013,VIS,11,식당/카페,NaN,NaN,N,110,2022-07-05 11:24:40,NaN


In [43]:
data['TRAVEL_LIKE_SGG_1'].value_counts()

TRAVEL_LIKE_SGG_1
50130    366
50110    283
42150    267
42210    152
26350    105
        ... 
47920      1
47720      1
26380      1
11530      1
44250      1
Name: count, Length: 178, dtype: int64

In [30]:
data['TRAVELER_ID'].value_counts()

TRAVELER_ID
g002178    1
g004485    1
g000312    1
g001580    1
g000498    1
          ..
g011098    1
g003403    1
g000789    1
g013061    1
g006402    1
Name: count, Length: 2560, dtype: int64

In [22]:
data['TRAVEL_ID'].str.split('_').apply(lambda x: x[1][0])

0       e
1       e
2       e
3       e
4       e
       ..
2555    g
2556    g
2557    g
2558    h
2559    h
Name: TRAVEL_ID, Length: 2560, dtype: object

In [26]:
import numpy as np

np.equal(data['TRAVEL_ID'].str.split('_').apply(lambda x: x[0]), data['TRAVEL_ID'].str.split('_').apply(lambda x: x[1][0])).sum()

2560

In [44]:
tc_sgg = pd.read_csv('./data/TL_csv/tc_sgg_시군구코드.csv')

tc_sgg

,SGG_CD,SGG_CD1,SGG_CD2,SGG_CD3,SGG_CD4,SIDO_NM,SGG_NM,DONG_NM,RI_NM
0,1100000000,11,NaN,NaN,NaN,서울특별시,NaN,NaN,NaN
1,1111000000,11,110.0,NaN,NaN,서울특별시,종로구,NaN,NaN
2,1111010100,11,110.0,101.0,0.0,서울특별시,종로구,청운동,NaN
3,1111010200,11,110.0,102.0,0.0,서울특별시,종로구,신교동,NaN
4,1111010300,11,110.0,103.0,0.0,서울특별시,종로구,궁정동,NaN
...,...,...,...,...,...,...,...,...,...
8070,5013025929,50,130.0,259.0,29.0,제주특별자치도,서귀포시,성산읍,신풍리
8071,5013025930,50,130.0,259.0,30.0,제주특별자치도,서귀포시,성산읍,신천리
8072,5013025931,50,130.0,259.0,31.0,제주특별자치도,서귀포시,성산읍,난산리
8073,5013031000,50,130.0,310.0,0.0,제주특별자치도,서귀포시,안덕면,NaN


In [63]:
tc_sgg['SGG_CD'] = tc_sgg['SGG_CD'].astype(str)

tc_sgg[tc_sgg['SGG_CD'].str.startswith('26500')]

,SGG_CD,SGG_CD1,SGG_CD2,SGG_CD3,SGG_CD4,SIDO_NM,SGG_NM,DONG_NM,RI_NM
681,2650000000,26,500.0,NaN,NaN,부산광역시,수영구,NaN,NaN
682,2650010100,26,500.0,101.0,0.0,부산광역시,수영구,망미동,NaN
683,2650010200,26,500.0,102.0,0.0,부산광역시,수영구,수영동,NaN
684,2650010300,26,500.0,103.0,0.0,부산광역시,수영구,민락동,NaN
685,2650010400,26,500.0,104.0,0.0,부산광역시,수영구,광안동,NaN
686,2650010500,26,500.0,105.0,0.0,부산광역시,수영구,남천동,NaN


In [23]:
import os

os.listdir('./data/bami/western/')

['tn_activity_consume_his_활동소비내역_G.csv',
 'tn_activity_his_활동내역_G.csv',
 'tn_adv_consume_his_사전소비내역_G.csv',
 'tn_companion_info_동반자정보_G.csv',
 'tn_lodge_consume_his_숙박소비내역_G.csv',
 'tn_move_his_이동내역_G.csv',
 'tn_mvmn_consume_his_이동수단소비내역_G.csv',
 'tn_tour_photo_관광사진_G.csv',
 'tn_traveller_master_여행객 Master_G.csv',
 'tn_travel_여행_G.csv',
 'tn_visit_area_info_방문지정보_G.csv']

In [11]:
import os
import pandas as pd
import glob

base_dir = './data/bami/'

file_patterns = ['tn_activity_consume_his_활동소비내역_',
                 'tn_activity_his_활동내역_',
                 'tn_adv_consume_his_사전소비내역_',
                 'tn_companion_info_동반자정보_',
                 'tn_lodge_consume_his_숙박소비내역_',
                 'tn_move_his_이동내역_',
                 'tn_mvmn_consume_his_이동수단소비내역_',
                 'tn_tour_photo_관광사진_',
                 'tn_traveller_master_여행객 Master_',
                 'tn_travel_여행_',
                 'tn_visit_area_info_방문지정보_']

# Create a dictionary to store combined dataframes
combined_dfs = {pattern: [] for pattern in file_patterns}

# Loop through each region and find files matching the patterns
for region in ['capital', 'eastern', 'western']:
    region_dir = os.path.join(base_dir, region)
    for pattern in file_patterns:
        for file_path in glob.glob(os.path.join(region_dir, pattern + '*.csv')):
            df = pd.read_csv(file_path)
            df['region'] = region
            combined_dfs[pattern].append(df)

# Concatenate dataframes for each pattern and display
for pattern, dfs in combined_dfs.items():
    if dfs:
        combined_df = pd.concat(dfs, ignore_index=True)
        combined_dfs[pattern] = combined_df
        print(f"Combined data for pattern: {pattern}")
        display(combined_dfs[pattern])  # Or use combined_df.to_csv(f'{pattern}_combined.csv', index=False)

Combined data for pattern: tn_activity_consume_his_활동소비내역_


,TRAVEL_ID,VISIT_AREA_ID,ACTIVITY_TYPE_CD,ACTIVITY_TYPE_SEQ,CONSUME_HIS_SEQ,CONSUME_HIS_SNO,PAYMENT_NUM,BRNO,STORE_NM,ROAD_NM_ADDR,LOTNO_ADDR,ROAD_NM_CD,LOTNO_CD,PAYMENT_DT,PAYMENT_MTHD_SE,PAYMENT_AMT_WON,PAYMENT_ETC,SGG_CD,region
0,e_e002053,2306030007,2,0,0,1,1,4.013801e+09,CU,경기 시흥시 비둘기공원7길 65,경기 시흥시 대야동 550-4,4397301.0,4.139010e+09,2023-06-03 23:01,1.0,6500.0,새우깡블랙;뉴리뉴멀티,NaN,capital
1,e_e001001,2306090003,1,0,0,1,2,1.338122e+09,가평(춘천)휴게소,경기 가평군 설악면 미사리로540번길 51,NaN,4448077.0,4.182031e+09,2023-06-09 20:16,1.0,5500.0,통감자,NaN,capital
2,e_e009156,2308260006,1,0,0,1,2,2.204101e+09,개항백화,인천 중구 개항로 104,인천 중구 경동 114,3149001.0,2.811013e+09,2023-08-26 15:42,1.0,22000.0,ICED 아메리카노;개항로 맥주,NaN,capital
3,e_e002077,2306010003,1,0,0,1,4,8.912202e+09,경주십원빵,인천 중구 영종진광장로 5,인천 중구 중산동 1944-5,3351842.0,2.811014e+09,2023-06-01 11:20,1.0,11000.0,NaN,NaN,capital
4,e_e001949,2306010005,1,0,0,1,2,6.323501e+09,단일서울,서울 성동구 왕십리로 66-33,서울 성동구 성수동1가 656-471,3005011.0,1.120011e+09,2023-06-01 13:13,1.0,11000.0,NaN,NaN,capital
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43927,h_h001459,2307060007,1,1,1,1,2,2.201901e+09,볼브,충북 단양군 단양읍 도전4길 2,충북 단양군 단양읍 도전리 448,4544127.0,4.380025e+09,2023-07-06 18:00,1.0,30500.0,땡모반;약과샌드;더블머그텀블러;대나무칫솔,NaN,western
43928,h_h001459,2307060010,2,1,1,1,2,2.261861e+09,CU,충북 단양군 단양읍 삼봉로 41,충북 단양군 단양읍 상진리 84-14,3248030.0,4.380025e+09,2023-07-06 20:30,1.0,7700.0,빙그레)캔디바;무파마큰사발컵;진짬뽕컵;코카콜라제로;친환경봉투판매용,NaN,western
43929,h_h003280,2308250004,1,0,0,1,1,1.751002e+09,옛고을,충북 보은군 속리산면 법주사로 250,충북 보은군 속리산면 사내리 280-1,3241010.0,4.372032e+09,2023-08-25 12:34,1.0,21000.0,산채돌솥비빔밥;도토리묵,NaN,western
43930,h_h003280,2308250005,1,0,0,1,1,2.598502e+09,문의(청주)휴게소,충북 청주시 상당구 문의면 당진영덕고속도로 123,충북 청주시 상당구 문의면 남계리 650-5,1000015.0,4.311135e+09,2023-08-25 13:41,1.0,8300.0,빅아메리카노(차가운);로띠번(오리지널),NaN,western


Combined data for pattern: tn_activity_his_활동내역_


,TRAVEL_ID,VISIT_AREA_ID,ACTIVITY_TYPE_CD,ACTIVITY_TYPE_SEQ,ACTIVITY_ETC,ACTIVITY_DTL,RSVT_YN,EXPND_SE,ADMISSION_SE,region
0,e_e000004,2304300002,3,0,NaN,화성 어차 탑승,Y,5,1.0,capital
1,e_e000004,2304300003,4,0,NaN,NaN,NaN,NaN,NaN,capital
2,e_e000004,2304300004,4,0,NaN,NaN,NaN,NaN,NaN,capital
3,e_e000006,2304300005,1,0,NaN,소금빵,N,1,NaN,capital
4,e_e000006,2304300007,1,0,NaN,쭈꾸미,N,1,NaN,capital
...,...,...,...,...,...,...,...,...,...,...
89809,h_h003280,2308250003,4,1,NaN,NaN,NaN,NaN,NaN,western
89810,h_h003280,2308250004,5,1,NaN,NaN,NaN,NaN,NaN,western
89811,h_h003280,2308250002,5,2,NaN,NaN,NaN,NaN,NaN,western
89812,h_h003280,2308250003,5,2,NaN,NaN,NaN,NaN,NaN,western


Combined data for pattern: tn_adv_consume_his_사전소비내역_


...

,TRAVELER_ID,RESIDENCE_SGG_CD,GENDER,AGE_GRP,EDU_NM,EDU_FNSH_SE,MARR_STTS,FAMILY_MEMB,JOB_NM,JOB_ETC,...,TRAVEL_STYL_8,TRAVEL_STATUS_RESIDENCE,TRAVEL_STATUS_DESTINATION,TRAVEL_STATUS_ACCOMPANY,TRAVEL_STATUS_YMD,TRAVEL_MOTIVE_1,TRAVEL_MOTIVE_2,TRAVEL_MOTIVE_3,TRAVEL_COMPANIONS_NUM,region
0,e004720,41,여,60,4,1.0,3,3,11,NaN,...,5,경기도,서울,2인 가족 여행,2023-07-16~2023-07-16,2,6.0,NaN,1,capital
1,e000914,30,여,20,6,1.0,1,1,3,NaN,...,1,대전광역시,서울,나홀로 여행,2023-06-03~2023-06-03,1,7.0,10.0,0,capital
2,e003564,41,여,30,7,1.0,2,4,2,NaN,...,7,경기도,경기,자녀 동반 여행,2023-06-24~2023-06-24,8,3.0,7.0,3,capital
3,e000396,41,여,30,6,1.0,2,2,2,NaN,...,6,경기도,인천,2인 가족 여행,2023-05-20~2023-05-21,9,1.0,7.0,1,capital
4,e001890,11,남,20,6,1.0,1,4,3,NaN,...,6,서울특별시,경기,2인 여행(가족 외),2023-06-04~2023-06-04,3,1.0,5.0,1,capital
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7675,g005855,26,여,20,6,1.0,1,4,3,NaN,...,4,부산광역시,전북,2인 여행(가족 외),2023-08-16~2023-08-17,7,3.0,2.0,1,western
7676,g012396,29,여,50,5,1.0,2,4,3,NaN,...,5,광주광역시,전남,자녀 동반 여행,2023-09-10~2023-09-10,3,1.0,2.0,1,western
7677,g012655,36,남,30,7,1.0,2,4,2,NaN,...,4,세종특별자치시,충북,자녀 동반 여행,2023-09-23~2023-09-23,7,3.0,1.0,3,western
7678,g012604,11,남,20,4,1.0,1,4,12,NaN,...,6,서울특별시,대전,2인 여행(가족 외),2023-09-16~2023-09-16,7,2.0,3.0,1,western


Combined data for pattern: tn_travel_여행_


,TRAVEL_ID,TRAVEL_NM,TRAVELER_ID,TRAVEL_PURPOSE,TRAVEL_START_YMD,TRAVEL_END_YMD,MVMN_NM,TRAVEL_PERSONA,TRAVEL_MISSION,TRAVEL_MISSION_CHECK,region
0,e_e000004,E03,e000004,3,2023-04-30,2023-05-01,NaN,서울 외 수도권 방문/수도권 거주/40세 이상/자녀동반/일반미션,3,3;4;11,capital
1,e_e000006,E03,e000006,21,2023-04-30,2023-05-02,NaN,경기 방문/거주지 구분 없음/39세 이하/특별미션,21,21;10;27,capital
2,e_e000009,E03,e000009,2;4,2023-04-29,2023-05-01,NaN,서울 외 수도권 방문/수도권 거주/39세 이하/커플/일반미션,2;4,22;1;7,capital
3,e_e000010,E01,e000010,3;6,2023-04-29,2023-05-01,NaN,서울 방문/수도권 외 거주/39세 이하/나홀로 여행/일반미션,3;6,6;3;1,capital
4,e_e000011,E01,e000011,1;21,2023-04-28,2023-05-01,NaN,서울 방문/수도권 외 거주/40세 이상/커플/일반미션,1;21,6;2;24,capital
...,...,...,...,...,...,...,...,...,...,...,...
7675,g_g013127,G06,g013127,1;10;2;23,2023-09-27,2023-09-28,NaN,충청권 방문/충청권 거주/39세 이하/나홀로 여행/일반미션,1;10;2;23,1;2;3,western
7676,g_g013128,G04,g013128,21;22;24;3;6,2023-09-29,2023-09-30,NaN,전남 방문/거주지 구분 없음/39세 이하/특별미션,21;22;24;3;6,6;1;21,western
7677,g_g013131,G03,g013131,2;21;22;26;4,2023-09-29,2023-09-30,NaN,전남 방문/거주지 구분 없음/39세 이하/특별미션,2;21;22;26;4,4;22;21,western
7678,h_h001459,G05,h001459,22,2023-07-06,2023-07-08,대중교통 등,충북 방문/거주지 구분 없음/39세 이하/특별미션,22,22;5;26,western


Combined data for pattern: tn_visit_area_info_방문지정보_


,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,VISIT_START_YMD,VISIT_END_YMD,ROAD_NM_ADDR,LOTNO_ADDR,X_COORD,Y_COORD,...,RESIDENCE_TIME_MIN,VISIT_AREA_TYPE_CD,REVISIT_YN,VISIT_CHC_REASON_CD,LODGING_TYPE_CD,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION,SGG_CD,region
0,2304300001,e_e000004,1,집,2023-04-30,2023-04-30,NaN,NaN,NaN,NaN,...,NaN,21,NaN,NaN,NaN,NaN,NaN,NaN,4.159012e+09,capital
1,2304300002,e_e000004,2,화성 관광열차 안내소 연무대 매표소,2023-04-30,2023-04-30,경기 수원시 팔달구 창룡대로103번길 20,경기 수원시 팔달구 매향동 3-32,127.023339,37.287878,...,60.0,2,N,10.0,NaN,4.0,3.0,4.0,NaN,capital
2,2304300003,e_e000004,3,창룡문,2023-04-30,2023-04-30,NaN,경기 수원시 팔달구 남수동,127.025143,37.287791,...,30.0,2,N,1.0,NaN,4.0,4.0,4.0,NaN,capital
3,2304300004,e_e000004,4,수원 화성 화홍문,2023-04-30,2023-04-30,NaN,경기 수원시 팔달구 북수동 9000-1,127.017626,37.287546,...,60.0,2,N,10.0,NaN,4.0,3.0,3.0,NaN,capital
4,2304300005,e_e000004,5,집,2023-04-30,2023-05-01,NaN,NaN,NaN,NaN,...,390.0,21,NaN,NaN,NaN,NaN,NaN,NaN,4.159012e+09,capital
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79160,2308250002,h_h003280,2,법주사,2023-08-25,2023-08-25,충북 보은군 속리산면 법주사로 405,충북 보은군 속리산면 사내리 209,127.832958,36.541721,...,90.0,2,N,1.0,NaN,5.0,5.0,5.0,NaN,western
79161,2308250003,h_h003280,3,정이품송,2023-08-25,2023-08-25,NaN,충북 보은군 속리산면 상판리 241,127.815581,36.517404,...,30.0,1,N,1.0,NaN,5.0,5.0,5.0,NaN,western
79162,2308250004,h_h003280,5,옛 고을 식당,2023-08-25,2023-08-25,충북 보은군 속리산면 법주사로 250,충북 보은군 속리산면 사내리 280-1,127.821492,36.530456,...,30.0,11,N,7.0,NaN,5.0,5.0,5.0,NaN,western
79163,2308250005,h_h003280,4,문의 청남대 휴게소 청주 방향,2023-08-25,2023-08-25,충북 청주시 상당구 문의면 당진영덕고속도로 123,충북 청주시 상당구 문의면 남계리 산 59-6,127.484472,36.544257,...,30.0,4,N,9.0,NaN,5.0,5.0,5.0,NaN,western


In [49]:
combined_dfs['tn_activity_consume_his_활동소비내역_'][0]

,TRAVEL_ID,VISIT_AREA_ID,ACTIVITY_TYPE_CD,ACTIVITY_TYPE_SEQ,CONSUME_HIS_SEQ,CONSUME_HIS_SNO,PAYMENT_NUM,BRNO,STORE_NM,ROAD_NM_ADDR,LOTNO_ADDR,ROAD_NM_CD,LOTNO_CD,PAYMENT_DT,PAYMENT_MTHD_SE,PAYMENT_AMT_WON,PAYMENT_ETC,SGG_CD,region
0,e_e002053,2306030007,2,0,0,1,1,4.013801e+09,CU,경기 시흥시 비둘기공원7길 65,경기 시흥시 대야동 550-4,4397301.0,4.139010e+09,2023-06-03 23:01,1,6500,새우깡블랙;뉴리뉴멀티,NaN,capital
1,e_e001001,2306090003,1,0,0,1,2,1.338122e+09,가평(춘천)휴게소,경기 가평군 설악면 미사리로540번길 51,NaN,4448077.0,4.182031e+09,2023-06-09 20:16,1,5500,통감자,NaN,capital
2,e_e009156,2308260006,1,0,0,1,2,2.204101e+09,개항백화,인천 중구 개항로 104,인천 중구 경동 114,3149001.0,2.811013e+09,2023-08-26 15:42,1,22000,ICED 아메리카노;개항로 맥주,NaN,capital
3,e_e002077,2306010003,1,0,0,1,4,8.912202e+09,경주십원빵,인천 중구 영종진광장로 5,인천 중구 중산동 1944-5,3351842.0,2.811014e+09,2023-06-01 11:20,1,11000,NaN,NaN,capital
4,e_e001949,2306010005,1,0,0,1,2,6.323501e+09,단일서울,서울 성동구 왕십리로 66-33,서울 성동구 성수동1가 656-471,3005011.0,1.120011e+09,2023-06-01 13:13,1,11000,NaN,NaN,capital
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11734,g_g010556,2309020008,1,0,0,1,2,4.763600e+09,선창마트,경기 시흥시 오이도로 179,경기 시흥시 정왕동 2006-22,3199037.0,4.139013e+09,2023-09-02 18:57,1,12600,켈리;포카칩어니언;바리스타,NaN,capital
11735,h_h003275,2308270002,3,0,0,1,4,1.358300e+09,용인농촌테마파크,경기 용인시 처인구 원삼면 농촌파크로 80,NaN,3203025.0,4.146134e+09,2023-08-27 10:50,1,2000,입장권,NaN,capital
11736,h_h003275,2308270002,3,1,1,1,4,1.358300e+09,용인농촌 테마파크,경기 용인시 처인구 원삼면 농촌파크로 80,NaN,3203025.0,4.146134e+09,2023-08-27 11:04,1,7000,NaN,NaN,capital
11737,h_h003275,2308270004,1,0,0,1,4,1.420762e+09,외할머니집,경기 용인시 처인구 양지면 중부대로 2545-3,경기 용인시 처인구 양지면 제일리 34-1,2012012.0,4.146136e+09,2023-08-27 13:16,1,31000,NaN,NaN,capital


In [2]:
file_patterns = ['활동소비내역',
                 '활동내역',
                 '사전소비내역',
                 '동반자정보',
                 '숙박소비내역',
                 '이동내역',
                 '이동수단소비내역',
                 '관광사진',
                 '여행객 Master',
                 '여행',
                 '방문지정보']

dict_values([[       TRAVEL_ID  VISIT_AREA_ID  ACTIVITY_TYPE_CD  ACTIVITY_TYPE_SEQ  \
0      e_e002053     2306030007                 2                  0   
1      e_e001001     2306090003                 1                  0   
2      e_e009156     2308260006                 1                  0   
3      e_e002077     2306010003                 1                  0   
4      e_e001949     2306010005                 1                  0   
...          ...            ...               ...                ...   
11734  g_g010556     2309020008                 1                  0   
11735  h_h003275     2308270002                 3                  0   
11736  h_h003275     2308270002                 3                  1   
11737  h_h003275     2308270004                 1                  0   
11738  h_h003275     2308270005                 3                  0   

       CONSUME_HIS_SEQ  CONSUME_HIS_SNO  PAYMENT_NUM          BRNO  \
0                    0                1            

In [ ]:
file_patterns = ['tn_activity_consume_his_활동소비내역_',
                 'tn_activity_his_활동내역_',
                 'tn_adv_consume_his_사전소비내역_',
                 'tn_companion_info_동반자정보_',
                 'tn_lodge_consume_his_숙박소비내역_',
                 'tn_move_his_이동내역_',
                 'tn_mvmn_consume_his_이동수단소비내역_',
                 'tn_tour_photo_관광사진_',
                 'tn_traveller_master_여행객 Master_',
                 'tn_travel_여행_',
                 'tn_visit_area_info_방문지정보_']

In [20]:
file_patterns[0].split('_')[-2]

'활동소비내역'

In [22]:
for idx, df in enumerate(combined_dfs.values()):
    file_name = file_patterns[idx].split('_')[-2]
    df.to_csv(f'./data/bami/total/{file_name}.csv', index=None)

In [64]:
for idx, df in enumerate(combined_dfs.values()):
    combined_dfs[0].to_csv(f'./data/bami/total/{file_patterns[idx]}.csv', index=None)

In [58]:
for df in combined_dfs.values():
    display(df[0])

,TRAVEL_ID,VISIT_AREA_ID,ACTIVITY_TYPE_CD,ACTIVITY_TYPE_SEQ,CONSUME_HIS_SEQ,CONSUME_HIS_SNO,PAYMENT_NUM,BRNO,STORE_NM,ROAD_NM_ADDR,LOTNO_ADDR,ROAD_NM_CD,LOTNO_CD,PAYMENT_DT,PAYMENT_MTHD_SE,PAYMENT_AMT_WON,PAYMENT_ETC,SGG_CD,region
0,e_e002053,2306030007,2,0,0,1,1,4.013801e+09,CU,경기 시흥시 비둘기공원7길 65,경기 시흥시 대야동 550-4,4397301.0,4.139010e+09,2023-06-03 23:01,1,6500,새우깡블랙;뉴리뉴멀티,NaN,capital
1,e_e001001,2306090003,1,0,0,1,2,1.338122e+09,가평(춘천)휴게소,경기 가평군 설악면 미사리로540번길 51,NaN,4448077.0,4.182031e+09,2023-06-09 20:16,1,5500,통감자,NaN,capital
2,e_e009156,2308260006,1,0,0,1,2,2.204101e+09,개항백화,인천 중구 개항로 104,인천 중구 경동 114,3149001.0,2.811013e+09,2023-08-26 15:42,1,22000,ICED 아메리카노;개항로 맥주,NaN,capital
3,e_e002077,2306010003,1,0,0,1,4,8.912202e+09,경주십원빵,인천 중구 영종진광장로 5,인천 중구 중산동 1944-5,3351842.0,2.811014e+09,2023-06-01 11:20,1,11000,NaN,NaN,capital
4,e_e001949,2306010005,1,0,0,1,2,6.323501e+09,단일서울,서울 성동구 왕십리로 66-33,서울 성동구 성수동1가 656-471,3005011.0,1.120011e+09,2023-06-01 13:13,1,11000,NaN,NaN,capital
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11734,g_g010556,2309020008,1,0,0,1,2,4.763600e+09,선창마트,경기 시흥시 오이도로 179,경기 시흥시 정왕동 2006-22,3199037.0,4.139013e+09,2023-09-02 18:57,1,12600,켈리;포카칩어니언;바리스타,NaN,capital
11735,h_h003275,2308270002,3,0,0,1,4,1.358300e+09,용인농촌테마파크,경기 용인시 처인구 원삼면 농촌파크로 80,NaN,3203025.0,4.146134e+09,2023-08-27 10:50,1,2000,입장권,NaN,capital
11736,h_h003275,2308270002,3,1,1,1,4,1.358300e+09,용인농촌 테마파크,경기 용인시 처인구 원삼면 농촌파크로 80,NaN,3203025.0,4.146134e+09,2023-08-27 11:04,1,7000,NaN,NaN,capital
11737,h_h003275,2308270004,1,0,0,1,4,1.420762e+09,외할머니집,경기 용인시 처인구 양지면 중부대로 2545-3,경기 용인시 처인구 양지면 제일리 34-1,2012012.0,4.146136e+09,2023-08-27 13:16,1,31000,NaN,NaN,capital


,TRAVEL_ID,VISIT_AREA_ID,ACTIVITY_TYPE_CD,ACTIVITY_TYPE_SEQ,ACTIVITY_ETC,ACTIVITY_DTL,RSVT_YN,EXPND_SE,ADMISSION_SE,region
0,e_e000004,2304300002,3,0,NaN,화성 어차 탑승,Y,5,1.0,capital
1,e_e000004,2304300003,4,0,NaN,NaN,NaN,NaN,NaN,capital
2,e_e000004,2304300004,4,0,NaN,NaN,NaN,NaN,NaN,capital
3,e_e000006,2304300005,1,0,NaN,소금빵,N,1,NaN,capital
4,e_e000006,2304300007,1,0,NaN,쭈꾸미,N,1,NaN,capital
...,...,...,...,...,...,...,...,...,...,...
23684,h_h003275,2308270005,3,4,NaN,자율주행차 타기,N,5,1.0,capital
23685,h_h003275,2308270003,4,1,NaN,NaN,NaN,NaN,NaN,capital
23686,h_h003275,2308270005,4,1,NaN,NaN,NaN,NaN,NaN,capital
23687,h_h003275,2308270003,5,2,NaN,NaN,NaN,NaN,NaN,capital


,TRAVEL_ID,ADV_NM,ADV_SEQ,PAYMENT_NUM,BRNO,STORE_NM,ROAD_NM_ADDR,LOTNO_ADDR,ROAD_NM_CD,LOTNO_CD,PAYMENT_DT,PAYMENT_MTHD_SE,PAYMENT_AMT_WON,PAYMENT_ETC,SGG_CD,region
0,e_e000004,연무대 어차,1,3,1.358216e+09,수원문화재단,경기 수원시 팔달구 행궁로 11,NaN,3176017.0,4.111512e+09,2023-04-30 14:15,3,9500,화성어차,NaN,capital
1,e_e000011,체험예약,0,4,NaN,후후라운지,옥인3길8,NaN,NaN,NaN,NaN,1,140000,NaN,1.111011e+09,capital
2,e_e000030,ITX열차표,1,1,3.148210e+09,한국철도공사,대전 동구 중앙로 240,NaN,3164033.0,3.011011e+09,2023-04-29 00:00,1,0,기차표,NaN,capital
3,e_e000030,ITX열차표,2,1,3.148210e+09,한국철도공사,대전 동구 중앙로 240,NaN,3164033.0,3.011011e+09,NaN,1,0,기차표,NaN,capital
4,e_e000037,음료,1,1,NaN,카페윤,서울 금천구 독산로50길 60-12,서울 금천구 시흥동 793,4151113.0,1.154510e+09,2023-04-30 18:14,1,9500,NaN,NaN,capital
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
620,e_e013055,커피,0,3,NaN,스타벅스,NaN,NaN,NaN,NaN,NaN,1,9500,NaN,4.121010e+09,capital
621,e_e013062,연극 비용,1,2,2.118842e+09,㈜티몬,서울 강남구 압구정로 118,서울 강남구 신사동 529-4,3122007.0,1.168011e+09,2023-09-04 00:00,1,20060,[몬스터데이]연극 슬기로운 신혼생활 관람권,NaN,capital
622,e_e013063,뮤지컬 티켓,0,2,NaN,인터파크티켓,131,NaN,NaN,NaN,NaN,1,322000,NaN,1.114016e+09,capital
623,f_f000041,식료품 및 간식,0,2,NaN,고덕 그라시움 GS 슈퍼,그라시움 아파트 상가내에 있는 GS 슈퍼,NaN,NaN,NaN,NaN,1,33720,NaN,1.174010e+09,capital


,COMPANION_SEQ,TRAVEL_ID,REL_CD,COMPANION_GENDER,COMPANION_AGE_GRP,COMPANION_SITUATION,region
0,2,e_e000004,2,1,1,1,capital
1,1,e_e000004,1,2,4,1,capital
2,1,e_e000006,5,1,3,3,capital
3,1,e_e000009,8,1,4,3,capital
4,1,e_e000011,7,2,6,3,capital
...,...,...,...,...,...,...,...
3532,1,g_g005600,2,2,1,1,capital
3533,1,g_g010556,8,1,4,1,capital
3534,1,h_h003275,1,2,5,1,capital
3535,3,h_h003275,2,2,1,1,capital


,TRAVEL_ID,LODGING_NM,LODGING_PAYMENT_SEQ,LODGING_TYPE_CD,RSVT_YN,CHK_IN_DT_MIN,CHK_OUT_DT_MIN,PAYMENT_NUM,BRNO,STORE_NM,ROAD_NM_ADDR,LOTNO_ADDR,ROAD_NM_CD,LOTNO_CD,PAYMENT_DT,PAYMENT_MTHD_SE,PAYMENT_AMT_WON,PAYMENT_ETC,region
0,e_e000013,해브펀캠핑어라운드,0,5,Y,NaN,NaN,4,NaN,NaN,경기 파주시 월롱면 도감로 130,경기 파주시 월롱면 위전리 89-1,NaN,NaN,NaN,1,240000,NaN,capital
1,e_e000020,파로티아워터하우스,1,1,Y,NaN,NaN,1,NaN,아고다,서울 중구 을지로 100,서울 중구 을지로2가 203,3101006.0,1.114010e+09,2023-04-29 00:00,1,84375,파로티아 워터하우스 호텔,capital
2,e_e000021,호텔헤르메스,1,1,N,NaN,NaN,2,NaN,화성 헤르메스 호텔,경기 화성시 남양읍 역골로 9-28,경기 화성시 남양읍 남양리 2076-17,3210080.0,4.159026e+09,2023-04-29 15:00,1,70000,숙박비,capital
3,e_e000038,호텔 더 메이,1,1,Y,NaN,NaN,2,NaN,주식회사 여기어때컴퍼니,서울 강남구 봉은사로 479,NaN,3122004.0,1.168010e+09,2023-04-27 00:20,1,66000,월문호텔더메이,capital
4,e_e000294,봉화회관,0,2,Y,NaN,NaN,2,NaN,NaN,경기 용인시 처인구 포곡읍 부곡로 21,경기 용인시 처인구 포곡읍 전대리 164-3,NaN,NaN,NaN,1,25000,NaN,capital
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
740,e_e012346,집,0,2,N,NaN,NaN,2,NaN,NaN,신정현대홈타운,NaN,NaN,NaN,NaN,4,0,NaN,capital
741,f_f000041,초코키즈풀빌라,0,3,Y,NaN,NaN,2,NaN,NaN,경기 가평군 가평읍 호반로 2083-23,경기 가평군 가평읍 이화리 88,NaN,NaN,NaN,1,379000,NaN,capital
742,f_f000458,마이다스호텔&리조트,1,1,Y,NaN,NaN,3,NaN,지마켓,서울 강남구 테헤란로 152,서울 강남구 역삼동 737,3122010.0,1.168010e+09,2023-05-27 00:00,1,38250,마이다스 호텔 앤 리조트,capital
743,f_f006622,알멕스랜드,1,5,Y,NaN,NaN,4,NaN,서울특별시체육회,서울 중랑구 망우로 182,서울 중랑구 상봉동 136-18,3005032.0,1.126010e+09,2023-07-05 15:58,3,30000,NaN,capital


...

,TRAVEL_ID,MVMN_SE,PAYMENT_SE,PAYMENT_SEQ,MVMN_SE_NM,RSVT_YN,PAYMENT_NUM,BRNO,STORE_NM,PAYMENT_DT,PAYMENT_MTHD_SE,PAYMENT_AMT_WON,PAYMENT_ETC,region
0,e_e002471,1,주유비,0,자가용(승용/승합/트럭 등등),N,3,NaN,NaN,NaN,NaN,40000,NaN,capital
1,e_e000006,1,주유비,0,자가용(승용/승합/트럭 등등),N,2,NaN,NaN,NaN,NaN,10000,NaN,capital
2,e_e000006,5,교통비,0,지하철,N,2,NaN,NaN,NaN,NaN,3600,NaN,capital
3,e_e000009,1,주유비,0,자가용(승용/승합/트럭 등등),N,2,NaN,NaN,NaN,NaN,30000,NaN,capital
4,e_e000009,1,통행료,0,자가용(승용/승합/트럭 등등),N,2,NaN,NaN,NaN,NaN,3800,NaN,capital
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4117,g_g007343,1,주유비,0,자가용(승용/승합/트럭 등등),N,1,NaN,NaN,NaN,NaN,5000,NaN,capital
4118,g_g007343,1,통행료,0,자가용(승용/승합/트럭 등등),N,1,NaN,NaN,NaN,NaN,10000,NaN,capital
4119,g_g010556,1,주유비,0,자가용(승용/승합/트럭 등등),N,2,NaN,NaN,NaN,NaN,50000,NaN,capital
4120,h_h003275,1,주유비,0,자가용(승용/승합/트럭 등등),N,4,NaN,NaN,NaN,NaN,30000,NaN,capital


,TRAVEL_ID,VISIT_AREA_ID,TOUR_PHOTO_SEQ,PHOTO_FILE_ID,PHOTO_FILE_NM,PHOTO_FILE_FRMAT,PHOTO_FILE_DT,PHOTO_FILE_SAVE_PATH,PHOTO_FILE_RESOLUTION,PHOTO_FILE_X_COORD,PHOTO_FILE_Y_COORD,VISIT_AREA_NM,region
0,e_e006895,2308120004,0,e00689501003p0001,e00689501003p0001.jpg,jpg,2023-08-12 14:33:11,/photo/,3024x3024,127.326530,37.543518,NaN,capital
1,e_e001026,2306040006,0,e00102601006p0001,e00102601006p0001.jpg,jpg,2023-06-04 18:48:38,/photo/,3024x4032,127.305188,37.338830,NaN,capital
2,g_g000018,2304290008,0,g00001801007p0001,g00001801007p0001.jpg,jpg,2023-04-29 15:18:16,/photo/,4032x3024,126.987250,37.280422,NaN,capital
3,f_f006622,2308130001,0,f00662202002p0001,f00662202002p0001.jpg,jpg,2023-08-13 08:01:20,/photo/,2560x1920,127.007629,38.075142,NaN,capital
4,f_f006366,2308180003,0,f00636601003p0001,f00636601003p0001.jpg,jpg,2023-08-18 15:03:22,/photo/,4032x2268,126.617394,37.169123,NaN,capital
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14622,e_e008178,2308220017,15,e00817801017p0016,e00817801017p0016.jpg,jpg,2023-08-22 17:07:52,/photo/,4000x3000,127.199309,37.291072,NaN,capital
14623,e_e009181,2308220009,16,e00918101009p0017,e00918101009p0017.jpg,jpg,2023-08-22 15:33:07,/photo/,4032x3024,126.596528,37.473386,NaN,capital
14624,e_e008178,2308220017,16,e00817801017p0017,e00817801017p0017.jpg,jpg,2023-08-22 17:08:03,/photo/,4000x3000,127.199629,37.291139,NaN,capital
14625,e_e008178,2308220017,19,e00817801017p0020,e00817801017p0020.jpg,jpg,2023-08-22 17:08:47,/photo/,4000x3000,127.199629,37.291139,NaN,capital


,TRAVELER_ID,RESIDENCE_SGG_CD,GENDER,AGE_GRP,EDU_NM,EDU_FNSH_SE,MARR_STTS,FAMILY_MEMB,JOB_NM,JOB_ETC,...,TRAVEL_STYL_8,TRAVEL_STATUS_RESIDENCE,TRAVEL_STATUS_DESTINATION,TRAVEL_STATUS_ACCOMPANY,TRAVEL_STATUS_YMD,TRAVEL_MOTIVE_1,TRAVEL_MOTIVE_2,TRAVEL_MOTIVE_3,TRAVEL_COMPANIONS_NUM,region
0,e004720,41,여,60,4,1.0,3,3,11,NaN,...,5,경기도,서울,2인 가족 여행,2023-07-16~2023-07-16,2,6.0,NaN,1,capital
1,e000914,30,여,20,6,1.0,1,1,3,NaN,...,1,대전광역시,서울,나홀로 여행,2023-06-03~2023-06-03,1,7.0,10.0,0,capital
2,e003564,41,여,30,7,1.0,2,4,2,NaN,...,7,경기도,경기,자녀 동반 여행,2023-06-24~2023-06-24,8,3.0,7.0,3,capital
3,e000396,41,여,30,6,1.0,2,2,2,NaN,...,6,경기도,인천,2인 가족 여행,2023-05-20~2023-05-21,9,1.0,7.0,1,capital
4,e001890,11,남,20,6,1.0,1,4,3,NaN,...,6,서울특별시,경기,2인 여행(가족 외),2023-06-04~2023-06-04,3,1.0,5.0,1,capital
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,e002684,11,여,60,8,5.0,2,4,2,NaN,...,1,서울특별시,경기,자녀 동반 여행,2023-07-08~2023-07-08,2,3.0,1.0,3,capital
2556,e011641,11,남,30,4,1.0,1,2,3,NaN,...,6,서울특별시,경기,2인 여행(가족 외),2023-08-27~2023-08-27,6,7.0,1.0,1,capital
2557,e003297,41,남,20,5,1.0,1,4,8,NaN,...,7,경기도,서울,나홀로 여행,2023-06-18~2023-06-18,1,8.0,7.0,0,capital
2558,e009006,11,여,20,6,1.0,1,1,12,NaN,...,5,서울특별시,인천,2인 여행(가족 외),2023-08-24~2023-08-24,1,2.0,3.0,1,capital


,TRAVEL_ID,TRAVEL_NM,TRAVELER_ID,TRAVEL_PURPOSE,TRAVEL_START_YMD,TRAVEL_END_YMD,MVMN_NM,TRAVEL_PERSONA,TRAVEL_MISSION,TRAVEL_MISSION_CHECK,region
0,e_e000004,E03,e000004,3,2023-04-30,2023-05-01,NaN,서울 외 수도권 방문/수도권 거주/40세 이상/자녀동반/일반미션,3,3;4;11,capital
1,e_e000006,E03,e000006,21,2023-04-30,2023-05-02,NaN,경기 방문/거주지 구분 없음/39세 이하/특별미션,21,21;10;27,capital
2,e_e000009,E03,e000009,2;4,2023-04-29,2023-05-01,NaN,서울 외 수도권 방문/수도권 거주/39세 이하/커플/일반미션,2;4,22;1;7,capital
3,e_e000010,E01,e000010,3;6,2023-04-29,2023-05-01,NaN,서울 방문/수도권 외 거주/39세 이하/나홀로 여행/일반미션,3;6,6;3;1,capital
4,e_e000011,E01,e000011,1;21,2023-04-28,2023-05-01,NaN,서울 방문/수도권 외 거주/40세 이상/커플/일반미션,1;21,6;2;24,capital
...,...,...,...,...,...,...,...,...,...,...,...
2555,g_g003294,E03,g003294,12;23;26;3;9,2023-07-08,2023-07-10,자가용,서울 외 수도권 방문/수도권 거주/39세 이하/부부/일반미션,12;23;26;3;9,12;3;23,capital
2556,g_g005600,E03,g005600,4;9,2023-08-23,2023-08-24,NaN,서울 외 수도권 방문/수도권 거주/39세 이하/자녀동반/일반미션,4;9,6;11;22,capital
2557,g_g007343,E03,g007343,1,2023-08-19,2023-08-20,NaN,서울 외 수도권 방문/수도권 외 거주/39세 이하/나홀로 여행/일반미션,1,1;22;26,capital
2558,g_g010556,E03,g010556,22,2023-09-02,2023-09-03,자가용,경기 방문/거주지 구분 없음/39세 이하/특별미션,22,22;5;26,capital


,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,VISIT_START_YMD,VISIT_END_YMD,ROAD_NM_ADDR,LOTNO_ADDR,X_COORD,Y_COORD,...,RESIDENCE_TIME_MIN,VISIT_AREA_TYPE_CD,REVISIT_YN,VISIT_CHC_REASON_CD,LODGING_TYPE_CD,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION,SGG_CD,region
0,2304300001,e_e000004,1,집,2023-04-30,2023-04-30,NaN,NaN,NaN,NaN,...,NaN,21,NaN,NaN,NaN,NaN,NaN,NaN,4.159012e+09,capital
1,2304300002,e_e000004,2,화성 관광열차 안내소 연무대 매표소,2023-04-30,2023-04-30,경기 수원시 팔달구 창룡대로103번길 20,경기 수원시 팔달구 매향동 3-32,127.023339,37.287878,...,60.0,2,N,10.0,NaN,4.0,3.0,4.0,NaN,capital
2,2304300003,e_e000004,3,창룡문,2023-04-30,2023-04-30,NaN,경기 수원시 팔달구 남수동,127.025143,37.287791,...,30.0,2,N,1.0,NaN,4.0,4.0,4.0,NaN,capital
3,2304300004,e_e000004,4,수원 화성 화홍문,2023-04-30,2023-04-30,NaN,경기 수원시 팔달구 북수동 9000-1,127.017626,37.287546,...,60.0,2,N,10.0,NaN,4.0,3.0,3.0,NaN,capital
4,2304300005,e_e000004,5,집,2023-04-30,2023-05-01,NaN,NaN,NaN,NaN,...,390.0,21,NaN,NaN,NaN,NaN,NaN,NaN,4.159012e+09,capital
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21379,2308270002,h_h003275,2,용인농촌테마파크,2023-08-27,2023-08-27,경기 용인시 처인구 원삼면 농촌파크로 80-1,경기 용인시 처인구 원삼면 사암리 871-3,127.298149,37.172338,...,60.0,13,N,1.0,NaN,4.0,4.0,4.0,NaN,capital
21380,2308270003,h_h003275,3,내동마을 연꽃 단지,2023-08-27,2023-08-27,경기 용인시 처인구 원삼면 내동로23번길 6,경기 용인시 처인구 원삼면 사암리 827,127.296806,37.175998,...,30.0,1,N,2.0,NaN,3.0,4.0,3.0,NaN,capital
21381,2308270004,h_h003275,4,외할머니 집,2023-08-27,2023-08-27,경기 용인시 처인구 양지면 중부대로 2545-3,경기 용인시 처인구 양지면 제일리 34-1,127.318988,37.242222,...,60.0,11,N,2.0,NaN,3.0,2.0,3.0,NaN,capital
21382,2308270005,h_h003275,5,삼성화재 모빌리티 뮤지엄,2023-08-27,2023-08-27,경기 용인시 처인구 포곡읍 에버랜드로376번길 171,경기 용인시 처인구 포곡읍 유운리 432,127.216930,37.300360,...,90.0,13,Y,4.0,NaN,5.0,5.0,5.0,NaN,capital


submodule TEST